In [1]:
import cv2
import numpy as np
import math
import time

In [2]:
capture=cv2.VideoCapture(0)
ret = capture.set(cv2.CAP_PROP_FRAME_HEIGHT,1080)
ret = capture.set(cv2.CAP_PROP_FRAME_WIDTH,1080)

TIMER = int(3)

while capture.isOpened():
    ret,frame=capture.read()
    cv2.line(frame,(650,0),(650,900),(255,255,255),2)
    cv2.rectangle(frame,(100,100),(500,500),(255,255,255),2)#Drawing the first rectangle for player 1
    cv2.rectangle(frame,(800,100),(1200,500),(255,255,255),2)#Drawing the second rectangle for player two
    cv2.putText(frame, "PLAYER 1", (50, 600), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
    cv2.putText(frame, "PLAYER 2", (700, 600), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
    cv2.imshow('GAME WINDOW',frame)
    
    k = cv2.waitKey(1)
    
    if k == ord('q'):
        prev = time.time()
 
        while TIMER >= 0:
            ret,frame = capture.read()
            cv2.line(frame,(650,0),(650,900),(255,255,255),2)
            cv2.rectangle(frame,(100,100),(500,500),(255,255,255),2)#Drawing the first rectangle for player 1
            cv2.rectangle(frame,(800,100),(1200,500),(255,255,255),2)#Drawing the second rectangle for player two
            cv2.putText(frame, "PLAYER 1", (50, 600), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
            cv2.putText(frame, "PLAYER 2", (700, 600), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
            cv2.imshow('GAME WINDOW',frame)
 
            # Display countdown on each frame
            # specify the font and draw the
            # countdown using puttext
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, str(TIMER),
                        (650,250), font,
                        5, (0, 255, 255),
                        4, cv2.LINE_AA)
            cv2.imshow('GAME WINDOW',frame)
            cv2.waitKey(125)
 
            # current time
            cur = time.time()
 
            # Update and keep track of Countdown
            # if time elapsed is one second
            # than decrease the counter
            if cur-prev >= 1:
                prev = cur
                TIMER = TIMER-1
                
        else:
            ret,frame = capture.read()
            cv2.line(frame,(650,0),(650,900),(255,255,255),2)
            cv2.rectangle(frame,(100,100),(500,500),(255,255,255),2)#Drawing the first rectangle for player 1
            cv2.rectangle(frame,(800,100),(1200,500),(255,255,255),2)#Drawing the second rectangle for player two
            cv2.putText(frame, "PLAYER 1", (50, 600), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
            cv2.putText(frame, "PLAYER 2", (700, 600), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
            cv2.imshow('GAME WINDOW',frame)
 
            # Display the clicked frame for 2
            # sec.You can increase time in
            # waitKey also
            cv2.imshow('GAME WINDOW',frame)
 
            # time for which image displayed
            cv2.waitKey(2000)
 
            # Save the frame
            img_name1="firstplayergesture.png"
            firstp=frame[100:500,100:500]
            cv2.imwrite(img_name1,firstp)
            secondp=frame[100:500,800:1200]
            img_name2="secondplayergesture.png"
            cv2.imwrite(img_name2,secondp)
            
            # removing noise
            firstp=cv2.GaussianBlur(firstp,(5,5),0)#filtering the first player gesture
            secondp=cv2.GaussianBlur(secondp,(5,5),0)#filtering the second player gesture
 
            # Reading the captured images
            hand1= cv2.imread('firstplayergesture.png')
            hand2= cv2.imread('secondplayergesture.png')
            
            lower_skin=np.array([0,30,60])
            upper_skin=np.array([20,150,255])
            
            # thresholding of first player gesture
            hsv1=cv2.cvtColor(hand1,cv2.COLOR_BGR2HSV)
            first_binary=cv2.inRange(hsv1,lower_skin,upper_skin)
          
            # threshholding of second player gesture
            hsv2=cv2.cvtColor(hand2,cv2.COLOR_BGR2HSV)
            second_binary=cv2.inRange(hsv2,lower_skin,upper_skin)
           
            
            # morphological transformations
            kernel=np.ones((3,3),np.uint8)#kernel for morphological transformations
    
            # morphological transformation for player one gesture
            dilution1=cv2.dilate(first_binary,kernel,iterations=1)
            erosion1=cv2.erode(dilution1,kernel,iterations=1)
    
            # morphological transformation for player 2 gesture
            dilution2=cv2.dilate(second_binary,kernel,iterations=1)
            erosion2=cv2.erode(dilution2,kernel,iterations=1)
                
            # Filtering again
            ret1,the1=cv2.threshold(erosion1,70,255,cv2.THRESH_BINARY)
            ret2,the2=cv2.threshold(erosion2,70,255,cv2.THRESH_BINARY)
            
            #To find outer area(Connected pixels) using contour
            contours1, hierarchy = cv2.findContours(the1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  
            contours2, heirarchy= cv2.findContours(the2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            
            #cv2.imshow("vyg",contours1)

            hull1 = [cv2.convexHull(c) for c in contours1]  
            hull2 = [cv2.convexHull(c) for c in contours2]

            #To check result we need to draw contour
            final1 = cv2.drawContours(hand1, hull1, -1, (255,0,0))
            final2 = cv2.drawContours(hand2, hull2, -1, (255,0,0))
    
            #Find convexity defects for both players
            hull1 = cv2.convexHull(contours1, returnPoints=False)
            defects1 = cv2.convexityDefects(contour1, hull1)
            hull2 = cv2.convexHull(contours2, returnPoints=False)
            defects2 = cv2.convexityDefects(contour2, hull2)
                
            # Use cosine rule to find angle of the far point from the start and end point i.e. the convex points (the finger
            # tips) for all defects
            all_image1 = np.hstack((final1, firstp))
            cv2.imshow('Contour for player 1', all_image1)
            all_image2 = np.hstack((final2, secondp))  #Horizontal stack-combines drawing2 and player 2 gesture
            cv2.imshow('Contours for player 2', all_image2)
                    
            count_defects = 0
            for i in range(defects1.shape[0]):
                s, e, f, d = defects1[i, 0]
                start = tuple(contour1[s][0])
                end = tuple(contour1[e][0])
                far = tuple(contour1[f][0])
            
                #If the angle of hand is changed
                a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14

                #if angle > 90 draw a circle at the far point
                if angle <= 90:
                    count_defects += 1
                    cv2.circle(firstp, far, 1, [0, 0, 255], -1)
                    cv2.line(firstp, start, end, [0, 255, 0], 2)
                        
                # Print the Gestures
            if count_defects == 0:
                cv2.putText(frame, "STONE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
            elif count_defects == 1:
                cv2.putText(frame, "Scissor", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
            elif count_defects == 4:
                cv2.putText(frame, "Paper", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2) 
            else:
                pass
    
    # Press Esc to exit
    elif k == 27:
        break
# close the camera
capture.release()

#cv2.waitKey(0)
  
# close all the opened windows
cv2.destroyAllWindows()

TypeError: Expected Ptr<cv::UMat> for argument 'points'